In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define a simple model
class HybridQCNN(nn.Module):
    def __init__(self):
        super(HybridQCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Change 1 to 3 for RGB images
        self.fc1 = nn.Linear(32 * 32 * 32, 64)  # Adjust the size according to your model
        self.fc2 = nn.Linear(64, 7)  # 7 classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(-1, 32 * 32 * 32)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize the model
model = HybridQCNN()

# Check parameters
print("Model Parameters:")
for param in model.parameters():
    print(param.size())

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data for testing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB images
])

dataset = datasets.ImageFolder(root='/kaggle/input/dental-radiography', transform=transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Training loop (simple example)
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(data_loader)}')

# Save the trained model
torch.save(model.state_dict(), 'model.pth')
print("Model saved as 'model.pth'")


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Define the model to match the saved state dict
class HybridQCNN(nn.Module):
    def __init__(self):
        super(HybridQCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # RGB images
        self.fc1 = nn.Linear(32 * 32 * 32, 64)  # Adjust according to saved model
        self.fc2 = nn.Linear(64, 7)  # 7 classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(-1, 32 * 32 * 32)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB images
])

# Load model
model = HybridQCNN()
try:
    model.load_state_dict(torch.load('model.pth'))
    print("Model loaded successfully.")
except RuntimeError as e:
    print(f"Error loading model: {e}")
model.eval()

# Map labels to symptoms
symptoms = [
    'stumps', 'implant teeth', 'missing teeth', 
    'patches', 'periodontal', 'impacted teeth', 
    'root canals'
]

def get_symptom(label):
    return symptoms[label]

def classify_image(image_path):
    # Load the image
    image = Image.open(image_path).convert("RGB")  # Adjusted to RGB if needed
    
    # Apply transformations
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Perform classification
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_label = predicted.item()
    
    return predicted_label

# Path to the single image
image_path = '/kaggle/input/dentalscan2/52.png'
label = classify_image(image_path)
symptom = get_symptom(label)
print(f'The image is classified as: {symptom}')
